### 1

In [1]:
install.packages(c("dplyr", "tidyr", "stringr"))
install.packages("dplyr")
install.packages("data.table")
install.packages("openxlsx")

Warning message in readRDS(dest):
"lzma decoding result 10"

package 'dplyr' successfully unpacked and MD5 sums checked
package 'tidyr' successfully unpacked and MD5 sums checked
package 'stringr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\dimab\AppData\Local\Temp\Rtmpk3enV4\downloaded_packages
package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\dimab\AppData\Local\Temp\Rtmpk3enV4\downloaded_packages

  There is a binary version available but the source version is later:
           binary source needs_compilation
data.table 1.13.6 1.14.0              TRUE

  Binaries will be installed


Warning message in download.file(url, destfile, method, mode = "wb", ...):
"загруженный размер 1556480 != сообщенному размеру 2549278"Warning message in unzip(zipname, exdir = dest):
"ошибка 1 при распаковке zip-файла"Warning message in read.dcf(file.path(pkgname, "DESCRIPTION"), c("Package", "Type")):
"не могу открыть сжатый файл 'data.table/DESCRIPTION', возможная причина -- 'No such file or directory'"

ERROR: Error in read.dcf(file.path(pkgname, "DESCRIPTION"), c("Package", "Type")): не могу открыть соединение


Загрузка данных для задания

In [ ]:
url <- "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
download_folder <- ".\\data_input\\"

In [ ]:
download.file(url, paste(download_folder, "time_series_covid19_confirmed_global.csv", sep=""))

In [ ]:
a <- read.csv(".\\data_input\\time_series_covid19_confirmed_global.csv")

In [ ]:
head(a)

### 2

"Выправление" названий столбцов с датами (*X* стоит в начале названия каждого столбца)

In [ ]:
cols <- colnames(a)[4:length(colnames(a))]
cols <- sapply(cols, function (x) return(sub("X", "", x)))
head(cols)

In [ ]:
colnames(a)[4:length(colnames(a))] <- cols 

In [ ]:
head(a)

### 3

In [ ]:
library(tidyr)
library(dplyr)

Объединение столбцов с названиями стран и провинций

In [ ]:
a <- unite(a, "Location", Province.State:Country.Region)
head(a)

И устранение "артефактов" их слияния

In [ ]:
a <- mutate(a, Location = gsub("^_", "", Location))
a <- mutate(a, Location = gsub("_", ": ", Location))

In [ ]:
head(a)

### 4

Тут создаётся фрейм с указанными в задании колонками, половина из которых вычисляется на основе данных о числе заболевших:

* Расположение;
* Широта;
* Долгота;
* Сумма заболевших;
* Среднее число заболевших;
* Стандартное отклонение числа заболевших

In [ ]:
second <- select(a, c("Location", "Lat", "Long"))
head(second)

In [ ]:
second["Amount"] <- rowSums(a[4:length(colnames(a))])
head(second)

In [ ]:
second["Average"] <- second["Amount"] / (length(colnames(a)) - 4)
head(second)

In [ ]:
second["SD"] <- apply(a[4:length(colnames(a))], 1, sd, na.rm=TRUE)
head(second)

### 5

Далее фрейм, по сути, просто транспонируется

In [ ]:
temp_a <- a[-(2:3)]
head(temp_a)

In [ ]:
result <- data.frame(t(temp_a[-1]))
colnames(result) <- temp_a[, 1]
head(result)

И первой колонке (индексной) задаётся название **Date**

In [ ]:
library(data.table)
setDT(result, keep.rownames="Date")
head(result)

Ну и дата приводится к нужному формату

In [ ]:
result$Date <- as.Date(result$Date, format="%m.%d.%y")
head(result)

### 6

Выводится итоговая табличка самыми изощрёнными способами

In [ ]:
dir.create(file.path("./", "data_output"), showWarnings=FALSE)

In [ ]:
write.table(result, "./data_output/result.txt")

In [ ]:
write.csv(result, "./data_output/result.csv", row.names=FALSE)

In [ ]:
library(openxlsx)
write.xlsx(result, "./data_output/result.xlsx")